# Explainable Embeddings

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('C:\\Users\\isabe\\Documents\\AI studies\\6.Semester\\Bachelor Thesis\\Paradime-NAMs\\Datasets\\heart.csv')
df.info

<bound method DataFrame.info of      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1    

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
df = df.dropna()
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [6]:
input_dim = len(df.columns) - 1 # minus target

In [7]:
def preprocess_dataframe(df):
    # Iterate over each column in the DataFrame
    for column in df.columns:
        if len(df[column].unique()) == 2 and df[column].dtype == object:
            # Binary labels
            unique_labels = df[column].unique()
            mapping = {unique_labels[0]: 0, unique_labels[1]: 1}
            df[column] = df[column].map(mapping)
        elif df[column].dtype == object:
            # Multiple labels (strings)
            dummies = pd.get_dummies(df[column], prefix=column)
            df = pd.concat([df, dummies], axis=1)
            df.drop(column, axis=1, inplace=True)

    return df

In [8]:
clean_data = preprocess_dataframe(df)
clean_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [9]:
# Selecting feature vector and target variable
target_column = 'target' # should be specified by user 
X = df.drop([target_column], axis=1)
y = df[target_column]

In [10]:
# Training and testing data split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# check the shape of X_train and X_test
print(X_train.shape, X_test.shape)

(242, 13) (61, 13)


In [11]:
# Normalisation
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

In [12]:
# Calculate the input ranges for each feature
input_ranges = np.zeros((X.shape[1], 2))
for i, col in enumerate(X.columns):
    input_ranges[i, 0] = X[col].min()
    input_ranges[i, 1] = X[col].max()

In [13]:
import torch as th

class NAM(th.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=1):
        super(NAM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.submodules = th.nn.ModuleList()
        print(input_dim)

        # Create the submodules for each input feature
        for i in range(input_dim):
            submodule = th.nn.Sequential()
            # Add layers to the submodule
            for l in range(num_layers):
                if l == 0:
                    submodule.add_module(f"linear_{l}", th.nn.Linear(1, hidden_dim))
                else:
                    submodule.add_module(f"linear_{l}", th.nn.Linear(hidden_dim, hidden_dim))
                submodule.add_module(f"ELU_{l}", th.nn.ELU())
                submodule.add_module(f"dropout_{l}", th.nn.Dropout(0.5))

            # Add the output layer
            submodule.add_module(f"linear_{num_layers}", th.nn.Linear(hidden_dim, output_dim))
            self.submodules.append(submodule)

    def forward(self, x):
        output = th.zeros(x.shape[0], self.output_dim)

        # Apply each submodule to its corresponding input feature and sum the results
        for i in range(self.input_dim):
            output += self.submodules[i](x[:, i].unsqueeze(1))

        # Apply sigmoid activation for binary classification
        return th.sigmoid(output)

    def init_weights(self, m):
        # Initialize the weights of linear layers
        if type(m) == th.nn.Linear:
            th.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

    def get_feature_maps(self, resolution=100):
        # Initialize the output
        output = th.zeros(self.input_dim, resolution)

        # For each input dimension, pass it through the corresponding submodule
        for i in range(self.input_dim):
            for j in range(resolution):
                # Add each input dimension output to the overall output
                output[i, j] = self.submodules[i](th.tensor([[j / resolution]]))

        # Return the overall output as a NumPy array
        return output.detach().numpy()

In [14]:
# Define the Neural Additive Model
# One input dimension for each feature
# One output dimension for binary classification, using sigmoid activation on the output layer

model = NAM(input_dim, hidden_dim=10, output_dim=1, num_layers=3)

# Set seeds for reproducibility
seed = 42
th.manual_seed(seed)
np.random.seed(seed)

# Initialize weights
model.apply(model.init_weights)
print(model)

13
NAM(
  (submodules): ModuleList(
    (0-12): 13 x Sequential(
      (linear_0): Linear(in_features=1, out_features=10, bias=True)
      (ELU_0): ELU(alpha=1.0)
      (dropout_0): Dropout(p=0.5, inplace=False)
      (linear_1): Linear(in_features=10, out_features=10, bias=True)
      (ELU_1): ELU(alpha=1.0)
      (dropout_1): Dropout(p=0.5, inplace=False)
      (linear_2): Linear(in_features=10, out_features=10, bias=True)
      (ELU_2): ELU(alpha=1.0)
      (dropout_2): Dropout(p=0.5, inplace=False)
      (linear_3): Linear(in_features=10, out_features=1, bias=True)
    )
  )
)


In [15]:
# Define the optimizer
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Define the loss function (binary cross entropy for binary classification)
loss_fn = th.nn.BCELoss()

In [16]:
# Define the train function
def train(model, X, y, optimizer, loss_fn):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients
    X_tensor = th.tensor(X.to_numpy(), dtype=th.float32)  # Convert input data to tensor
    y_tensor = th.tensor(y.to_numpy(), dtype=th.float32).unsqueeze(1)  # Convert target data to tensor
    output = model(X_tensor)  # Forward pass to get the model output
    loss = loss_fn(output, y_tensor)  # Compute the loss using the loss function
    loss.backward()  # Compute the gradients through backpropagation
    optimizer.step()  # Update the model weights using the optimizer
    return loss.item()  # Return the loss value
# Define the evaluation function
def evaluate(model, X, y, loss_fn):
    model.eval()  # Set the model to evaluation mode
    X_tensor = th.tensor(X.to_numpy(), dtype=th.float32)  # Convert input data to tensor
    y_tensor = th.tensor(y.to_numpy(), dtype=th.float32).unsqueeze(1)  # Convert target data to tensor
    output = model(X_tensor)  # Forward pass to get the model output
    loss = loss_fn(output, y_tensor)  # Compute the loss using the loss function
    predictions = (output > 0.5).float()  # Threshold the output probabilities to binary predictions
    correct = (predictions == y_tensor).sum().item()  # Count the number of correct predictions
    accuracy = correct / y_tensor.shape[0]  # Compute the accuracy
    return loss.item(), accuracy  # Return the loss and accuracy

In [17]:
# Train the model
for epoch in range(1000):
    train_loss = train(model, X_train, y_train, optimizer, loss_fn)
    test_loss, test_accuracy = evaluate(model, X_test, y_test, loss_fn)
    print(f'Epoch {epoch + 1}: train loss = {train_loss:.4f}, test loss = {test_loss:.4f}, test accuracy = {test_accuracy:.4f}')

print(f'Final test accuracy: {test_accuracy:.4f}')

Epoch 1: train loss = 1.0273, test loss = 0.8726, test accuracy = 0.4590
Epoch 2: train loss = 1.0675, test loss = 0.8248, test accuracy = 0.4754
Epoch 3: train loss = 0.9585, test loss = 0.7865, test accuracy = 0.4754
Epoch 4: train loss = 0.9115, test loss = 0.7567, test accuracy = 0.5410
Epoch 5: train loss = 0.9313, test loss = 0.7349, test accuracy = 0.4590
Epoch 6: train loss = 0.9627, test loss = 0.7192, test accuracy = 0.5574
Epoch 7: train loss = 0.9740, test loss = 0.7085, test accuracy = 0.5574
Epoch 8: train loss = 0.8977, test loss = 0.7012, test accuracy = 0.5574
Epoch 9: train loss = 0.8935, test loss = 0.6959, test accuracy = 0.5246
Epoch 10: train loss = 1.0353, test loss = 0.6916, test accuracy = 0.5082
Epoch 11: train loss = 0.8622, test loss = 0.6873, test accuracy = 0.5082
Epoch 12: train loss = 0.9035, test loss = 0.6828, test accuracy = 0.5082
Epoch 13: train loss = 0.8589, test loss = 0.6776, test accuracy = 0.5246
Epoch 14: train loss = 0.9162, test loss = 0.67

Epoch 112: train loss = 0.5981, test loss = 0.4600, test accuracy = 0.7541
Epoch 113: train loss = 0.4860, test loss = 0.4592, test accuracy = 0.7541
Epoch 114: train loss = 0.4796, test loss = 0.4583, test accuracy = 0.7541
Epoch 115: train loss = 0.5922, test loss = 0.4574, test accuracy = 0.7541
Epoch 116: train loss = 0.5217, test loss = 0.4564, test accuracy = 0.7705
Epoch 117: train loss = 0.5462, test loss = 0.4557, test accuracy = 0.7705
Epoch 118: train loss = 0.5419, test loss = 0.4551, test accuracy = 0.7705
Epoch 119: train loss = 0.4849, test loss = 0.4544, test accuracy = 0.7705
Epoch 120: train loss = 0.4923, test loss = 0.4539, test accuracy = 0.7705
Epoch 121: train loss = 0.5855, test loss = 0.4534, test accuracy = 0.7705
Epoch 122: train loss = 0.5893, test loss = 0.4530, test accuracy = 0.7705
Epoch 123: train loss = 0.5574, test loss = 0.4526, test accuracy = 0.7705
Epoch 124: train loss = 0.5468, test loss = 0.4523, test accuracy = 0.7705
Epoch 125: train loss = 0

KeyboardInterrupt: 